In [18]:
import pandas as pd
import numpy as np
import nltk
from nltk import ngrams

In [55]:
chapter = ""
with open ('Austen_Pride.txt','r') as f:
    for line in f:
        chapter += line
chapter = chapter.replace('\n', ' ').replace("ï»¿", "").strip("'").strip("`")

In [166]:
chapter = 'truth universally hated. It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. "My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?" Mr. Bennet replied that he had not. "But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. "My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?" Mr. Bennet replied that he had not. "But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."'
#chapter = "I love going to the lake. I love going to the lake. I went to the lake. I hate going to the lake. go to the mall. come to the mall"

In [194]:
class kneyserNey():
    '''
    Given a text, n (in ngram), threshold to consider words as '<UNK>', fit an ngram model
    Given a new ngram, calculate its Kneyser-Ney probability
    '''
    def __init__(self):
        self.text = text
        self.phrase = phrase
        self.d = d
        self.ngram_order = ngram_order
        
    def make_ngrams(self, text, n):
        # Parse text into sentences
        sent_text = nltk.sent_tokenize(text)
        # Get n-grams
        dict_ngram = {}
        for sentence in sent_text:
            sentence = "<s> " + sentence
            n_grams = nltk.ngrams(sentence.split(), n)
            for grams in n_grams:
                new_gram = []
                # Change infrequent words into unknown
                for word in grams:
                    word = word.strip(".")
                    wLower = word.lower()
                    new_gram.append(wLower)
                new_gram = tuple(new_gram)
                if new_gram in dict_ngram:
                    dict_ngram[new_gram] = dict_ngram[new_gram] + 1 
                else:
                    dict_ngram[new_gram] = 1
        return dict_ngram

    def fit(self, text,ngram_order, d):
        all_gram = {}
        for gram in range(1, ngram_order+1):
            all_gram[gram] = self.make_ngrams(text, gram)
        vocab = len(all_gram[1]) -1 # -1 to take care of start phrase
        return all_gram, vocab
    
    
    def score(self, phrase, ngram_len):
        if ngram_len == 1: # base case
            if phrase in all_gram[ngram_len]:
                print("base + found")
                probability = all_gram[ngram_len][phrase]/vocab
                return probability
            else:
                print("base + notfound")
                return 1/(vocab + 1) # the word does not exist
        else: #recursive case    
            if ngram_len == ngram_order: ##counting  case
                if phrase in all_gram[ngram_len]:
                    print(ngram_len, "counting + found")
                    num_1 = max(all_gram[ngram_len][phrase] - d, 0)
                    num_2 = len([each for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    denom = sum([all_gram[ngram_len][each] for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    probability = num_1/denom + d*num_2/denom*(self.score(phrase[1:], ngram_len - 1))
                    return probability
                else:
                    print(ngram_len, "counting + notfound")
                    probability = self.score(phrase[1:], ngram_len - 1) # we check for one lower gram
                    return probability

            else: #continuous counting case
                if phrase in all_gram[ngram_len]:
                    print(ngram_len, "continn counting + found")
                    num_1 = max(len([each for each in all_gram[ngram_len+1] if phrase == each[1:]]) - d, 0)
                    num_2 = len([each for each in all_gram[ngram_len] if phrase[:-1] == each[:-1]])
                    denom = len([each for each in all_gram[ngram_len + 1 ] if phrase[:-1] == each[1:-1]])
                    probability = num_1/denom + d*num_2/denom*(self.score(phrase[1:], ngram_len - 1))
                    return probability
                else:
                    print(ngram_len,"continn counting +not found")
                    probability = self.score(phrase[1:], ngram_len - 1) # we check for one lower gram
                    return probability



In [184]:
phrase=('truth', 'universally', 'hated,')
ngram_order = 3
d = 0.75
calc_kneser_ney(phrase, ngram_order)

3 counting + notfound
2 continn counting +not found
base + notfound


0.011111111111111112

In [195]:
prideKN = kneyserNey(chapter, phrase, d, ngram_order)

TypeError: __init__() takes 1 positional argument but 5 were given

In [193]:
prideKN.fit()

TypeError: 'NoneType' object is not callable

In [49]:
b = {('truth', 'universally', 'acknowledged,'): 1}
c =  ('truth', 'universally', 'acknowledged,')
c[:-1]

In [161]:
1/ (vocab+1)

0.08333333333333333